In [2]:
import os
import pandas as pd
import numpy as np
import urllib2
from bs4 import BeautifulSoup

data_location = os.path.join("..","data","external")

# Read in info needed for strings we need to search urls for
file_path = os.path.join(data_location, "MCCVB-locations-members.xlsx")
locations = pd.read_excel(file_path, sheetname = "locations")

# Read in urls data
file_path = os.path.join(data_location, "FY 16 - 17 PR Report Monthly Master.xlsx")
month_sheet = pd.read_excel(file_path, sheetname = "October 2016")

#Headers
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}


month_sheet_v1 = month_sheet[~pd.isnull(month_sheet['URL'])]
month_sheet_v1 = month_sheet_v1[~(month_sheet_v1['URL'].str.contains('n/a'))] 
month_sheet_v1 = month_sheet_v1[~(month_sheet_v1['URL'].str.contains('burrellesluce.com'))]
urls = month_sheet_v1['URL']
# select 10 for simplicity

def processUrl(url):
    query = urllib2.Request(url, None, hdr)
    html = urllib2.urlopen(query).read()
    soup = BeautifulSoup(html)
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text(separator=' ')
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

row = {}
i=0
for url in urls:
    i+= 1
    print "URL %s :: Processing: %s" % (i, url)

    try:
        text = processUrl(url)
    except:
        print "Please check the following URL it was not processed: %s " % (url)
    location_hits = []
    location_hits_string = ''

    
    
    if len(text) < 5:
        location_hits.append('No locations found, please check')
    else:
        for index, location in locations.iterrows():
            if location[0] in text:
                print location[0]
                location_hits.append(location[0])

            
            
            
    location_hits_string = ','.join(location_hits)
    url = str(url)
    row[url] = location_hits_string


frame = pd.DataFrame(row.items(), columns = ['URL','Juris_mention_auto'])
frame_out = pd.merge(month_sheet, frame, on = ['URL'], how = 'left')

writer = pd.ExcelWriter('FY 16-17 PR TESTING.xlsx')
frame_out.to_excel(writer, 'October 2016', index = False)

URL 1 :: Processing: http://www.traveldailynews.com/post/monterey-county-sees-unprecedented-levels-of-investment-and-a-wave-of-renovations
City of Monterey
Monterey
Monterey County
Marina
Pacific Grove
Seaside
Carmel
Carmel Valley
Big Sur
URL 2 :: Processing: http://markets.financialcontent.com/fatpitch.valueinvestingnews/news/read/32918982
Monterey
Monterey County
Pacific Grove
Salinas
Carmel
Big Sur
URL 3 :: Processing: https://www.equities.com/news/from-sea-to-table-monterey-county-restaurants-serving-up-sustainable-fare
Monterey
Monterey County
Pacific Grove
Salinas
Carmel
Big Sur
URL 4 :: Processing: http://seekingalpha.com/pr/16625837-monterey-county-sees-unprecedented-levels-investment-wave-renovations
City of Monterey
Monterey
Monterey County
Marina
Pacific Grove
Seaside
Carmel
Carmel Valley
Big Sur
URL 5 :: Processing: http://www.marketwatch.com/story/monterey-county-sees-unprecedented-levels-of-investment-and-a-wave-of-renovations-2016-10-05
City of Monterey
Monterey
Monterey

In [12]:
frame_out

,TYPE,PUB DATE,HEADLINE,PUBLICATION,PR MEDIA VALUE,CITY,STATE,IMPRESSIONS,COUNTRY,URL,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Juris_mention_auto
0,Online,2016-10-07,Monterey County See Unprecedented Levels of In...,Travel Daily News,n/a,n/a,n/a,423,US,http://www.traveldailynews.com/post/monterey-c...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"City of Monterey,Monterey,Monterey County,Mari..."
1,Online,2016-09-26,From Sea to Table: Monterey County Restaurants...,Value Investing News,n/a,n/a,n/a,4785,US,http://markets.financialcontent.com/fatpitch.v...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Monterey,Monterey County,Pacific Grove,Salinas..."
2,Online,2016-09-26,From Sea to Table: Monterey County Restaurants...,Equities.com,n/a,n/a,n/a,1368,US,https://www.equities.com/news/from-sea-to-tabl...,...,NaN,Total Ad Eq.,Total Impressions,NaN,NaN,NaN,NaN,NaN,NaN,"Monterey,Monterey County,Pacific Grove,Salinas..."
3,Online,2016-10-05,Monterey County See Unprecedented Levels of In...,Seeking Alpha,"72, 497",n/a,n/a,1105978,US,http://seekingalpha.com/pr/16625837-monterey-c...,...,NaN,1.28541e+06,74384010,NaN,NaN,NaN,NaN,NaN,NaN,"City of Monterey,Monterey,Monterey County,Mari..."
4,Online,2016-10-05,Monterey County See Unprecedented Levels of In...,MarketWatch,173121,NEW YORK,NY,3872649,US,http://www.marketwatch.com/story/monterey-coun...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"City of Monterey,Monterey,Monterey County,Mari..."
5,Online,2016-10-05,Monterey County See Unprecedented Levels of In...,The Business Journals,71095,CHARLOTTE,NC,"2,669, 960",US,http://www.bizjournals.com/prnewswire/press_re...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"No locations found, please check"
6,Online,2016-10-06,16 Amazing Places to Eat and Drink Near Monter...,Easter SF,64174,SAN FRANCISCO,CA,69659,US,http://sf.eater.com/maps/best-food-drinks-mont...,...,NaN,Jurisdiction Ad. Eq.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Monterey,Pacific Grove,Seaside,Carmel,Carmel V..."
7,Print,2016-10-01,DIVINE INSPIRATION,SMART MEETINGS,54695.1,SAUSALITO,CA,44100,US,http://images.burrellesluce.com/image/28287/28...,...,NaN,County,Monterey,Marina,Seaside,Pacific Grove,Salinas,Carmel,NaN,NaN
8,Print,2016-10-01,2016 Suppliers of the Year,SmartMeetings,40000,SAUSALITO,CA,1508,US,http://pubs.royle.com/publication/frame.php?i=...,...,NaN,1.19924e+06,1.19092e+06,882237,64174,1.11961e+06,0,1.13338e+06,NaN,
9,Online,2016-10-05,Monterey County See Unprecedented Levels of In...,EDGE Portland,33600,PORTLAND,OR,70000,US,http://portland.edgemedianetwork.com/prnewswir...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
